In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from preprocess import GSE
from models import AutoEncoder, VariationalAutoEncoder, PlotLatentSpace

In [2]:
dataset = GSE()
X_train, y_train = dataset.train
X_test, y_test = dataset.test

In [3]:
model = VariationalAutoEncoder(original_dim=dataset.n_genes)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=20)
plot_latent = PlotLatentSpace(model, X_train, y_train, interval=20)
model.compile(optimizer=Adam(0.0001), loss=model.reconstruction_loss)
history = model.fit(X_train, X_train, epochs=1000, validation_data=(X_test, X_test),
                    callbacks=[early_stopping, plot_latent], verbose=True)

Train on 322 samples, validate on 108 samples
Epoch 1/1000
320/322 [============================>.] - ETA: 0s - loss: 4010.4907WARNING:tensorflow:Layer Encoder is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

322/322 [==============================] - 8s 26ms/sample - loss: 4009.2230 - val_loss: 3731.3280
Epoch 2/1000
256/322 [======================>.......] - ETA: 0s 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
mu, logvar = model.encoder(X_train)
z = model.sampling([mu, logvar])

In [ ]:
from sklearn.manifold import TSNE
z_tsne = TSNE().fit_transform(z.numpy())
x_tsne = TSNE().fit_transform(X_train)
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
ax[0].scatter(z_tsne[:, 0], z_tsne[:, 1], c=y_train, cmap='rainbow')
ax[1].scatter(x_tsne[:, 0], x_tsne[:, 1], c=y_train, cmap='rainbow')

In [ ]:
x_hat = model.predict(X_train)
sns.heatmap(x_hat)